In [1]:
from pororo import Pororo
import pandas as pd
import torch
from tqdm import tqdm

/home/piai/anaconda3/envs/use_pororo/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
origin_strongbuy = pd.read_csv("./data/strongbuy.csv")
origin_sell = pd.read_csv("./data/sell.csv")
origin_holddown = pd.read_csv("./data/holddown.csv")

In [3]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [4]:
strongbuy_articles = list(origin_strongbuy["article"])
sell_articles = list(origin_sell["article"])
holddown_articles = list(origin_holddown["article"])

In [5]:
def max_len_slicing(input_str):
    return input_str[:1021]

# 길이제한 1024 - token 3개 빼고 1021
sliced_strongbuy = list(map(max_len_slicing, strongbuy_articles))
sliced_sell = list(map(max_len_slicing, sell_articles))
sliced_holddown = list(map(max_len_slicing, holddown_articles))

In [6]:
mt = Pororo(task="translation", lang="multi")

def back_trans_pororo(original_text_list, lang='en'):
    new_text_list = list()
    for original_text in tqdm(original_text_list):
        text_to_lang = mt(original_text, src="ko", tgt=lang)
        new_text = mt(text_to_lang, src=lang, tgt="ko")
        new_text_list.append(new_text)
    return new_text_list

In [7]:
expand_strongbuy_list = back_trans_pororo(sliced_strongbuy)
expand_sell_list = back_trans_pororo(sliced_sell)
expand_holddown_list = back_trans_pororo(sliced_holddown)

100%|███████████████████████████████████████| 830/830 [2:08:59<00:00,  9.32s/it]


In [8]:
origin_strongbuy["article"] = expand_strongbuy_list
origin_strongbuy.to_csv(f"./data/strongbuy_expand_pororo_en.csv", index=False)

origin_sell["article"] = expand_sell_list
origin_sell.to_csv(f"./data/sell_expand_pororo_en.csv", index=False)

origin_holddown["article"] = expand_holddown_list
origin_holddown.to_csv(f"./data/holddown_expand_pororo_en.csv", index=False)